##### Multithreading

##### Multiprocessing

##### GPU parallelism

### CPUs

The modern central processing unit and how amazing it is

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark sort(data) setup=(data=rand(10))

BenchmarkTools.Trial: 10000 samples with 792 evaluations.
 Range (min … max):  149.148 ns …   3.511 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     254.472 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   258.862 ns ± 113.454 ns  ┊ GC (mean ± σ):  1.06% ± 3.92%

                ▃██▃                                             
  ▂▂▃▄▄▃▃▄▃▃▂▂▂▄████▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  149 ns           Histogram: frequency by time          554 ns <

 Memory estimate: 144 bytes, allocs estimate: 1.

### Serial Processing

##### Example 1 

In [ ]:
function f(a, b)
    x = 0
    y = 0
    for i in 1:1
        x += a *i
        y += b*i
    end
    
    return x, y
end

f (generic function with 1 method)

##### Example 2

In [ ]:
using LinearAlgebra, BenchmarkTools

In [ ]:
function expensive()
    X = randn(10_000, 100)
    return inv(X'X)
end

expensive (generic function with 1 method)

In [ ]:
function go()
    A = expensive()
    B = expensive()
    return A + B
end

go (generic function with 1 method)

In [ ]:
@benchmark go()

BenchmarkTools.Trial: 81 samples with 1 evaluation.
 Range (min … max):  26.326 ms … 316.604 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     44.462 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   64.135 ms ±  55.133 ms  ┊ GC (mean ± σ):  3.64% ± 7.11%

  ▁▂█                                                           
  ████▆█▃▇▄▅▄▄▃▄▄▃▁▁▁▁▁▃▁▁▁▁▁▁▃▁▃▃▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▃ ▁
  26.3 ms         Histogram: frequency by time          290 ms <

 Memory estimate: 15.74 MiB, allocs estimate: 20.

### Multiprocess & multithread parallelism

In [ ]:
@benchmark randn(10_000, 10000)

InterruptException: InterruptException:

### Starting Julia with threads


In [ ]:
function heavy_shit()
    sleep(5)
    return dot(randn(3, 4), randn(4, 3))
end

task = Threads.@spawn heavy_shit()

println("waiting...")

# println(fetch(task))

waiting...


In [ ]:
task = Threads.@spawn heavy_shit()

Task (runnable) @0x00000001520ef6b0

##### Example 1

In [ ]:
function heavy_shit()
    return 0
end

heavy_shit (generic function with 1 method)

In [ ]:
heavy_shit

heavy_shit (generic function with 1 method)

Execute function `heavy_shit` on a thread, and assign to variable named `task`

In [ ]:
task = Threads.@spawn heavy_shit()

Task (done) @0x0000000155345430

In [ ]:
task

Task (done) @0x0000000155345430

##### Example 2

In [ ]:
Threads.nthreads()

10

##### Example 3

In [ ]:
function thread_test()
    Threads.@threads for i in 1:6
        id = Threads.threadid()
        println("Thread ID: ", id, " | i: ", i)
    end 
end

thread_test()

Thread ID: 10 | i: 5
Thread ID: 4 | i: 2
Thread ID: 3 | i: 3
Thread ID: 2 | i: 1
Thread ID: 8 | i: 4
Thread ID: 9 | i: 6


In [ ]:
Threads.threadid()

1

In [ ]:
function serialsum(xs)
    r = 0
    for i in xs
        r += i
    end
    return r
end

function threadsum(xs)
    r = zeros(Threads.nthreads())
    Threads.@threads for i in xs
        # Each thread has it's own "sum bucket" to accumulate 
        # bits and pieces of xs -- we'll add these buckets together
        # at the end.
        r[Threads.threadid()] += i
    end
    return sum(r)
end

values = randn(10_000);

In [ ]:
using BenchmarkTools
@benchmark serialsum(values)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  14.542 μs …  1.190 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     28.459 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   31.177 μs ± 32.621 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆    █▄▁ ▇▁   █▂                                            ▂
  █▃▂▄▅███▇██████████▆▇▇▇▇▇▇▇▇▆▆▆▆▇▅▅▅▅▅▅▄▄▄▄▅▄▅▄▅▂▄▃▃▃▄▃▃▂▄▄ █
  14.5 μs      Histogram: log(frequency) by time       105 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

##### Example

In [ ]:
xs = []

Any[]

In [ ]:
for i in 1:4
    println(i)
end

1
2
3
4


Make the loop run in multi-thread

In [ ]:
Threads.@threads for i in 1:4
    println(i)
end

3
4
2
1


### Even more complexity, race conditions!

### Locks

##### Example 1

In [21]:
xs = []

Any[]

In [22]:
numbers = 1:4

1:4

In [23]:
xs, numbers

(Any[], 1:4)

Do a `for loop` through `numbers` and add each number to the end of `xs` using **parallel with a lock**

In [24]:
xslock = Threads.ReentrantLock()

Threads.@threads for i in numbers
    lock(xslock) do
        push!(xs, i) # Add i to the end of xs
    end
end

In [25]:
xs

4-element Vector{Any}:
 1
 2
 3
 4

### Multiprocessing

In [2]:
function show_n_processor()
    return Distributed.nprocs()
end

show_n_processor (generic function with 1 method)

##### Example 1

In [1]:
using Distributed

In [3]:
show_n_processor()

1

Add for 4 new processors

In [4]:
Distributed.addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

In [5]:
show_n_processor()

5

##### Example 2

In [24]:
using Distributed

Make processor `1` do `1+1` and return the result to variable `result`

In [25]:
task = Distributed.@spawnat 1 1+1

Future(1, 1, 4, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (8, 5293285536, 0)), nothing)

In [26]:
result = fetch(task)

2

In [27]:
result

2

##### Example 3

In [12]:
using Distributed

Create a function add two numbers and execute that function to calculate `1 + 2` on processor `1`, stores the result in variable `result`

In [13]:
@everywhere add(x, y) = x + y

In [14]:
task = Distributed.@spawnat 1 add(1, 2)

Future(1, 1, 2, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 4338401288, 21474836481)), nothing)

In [15]:
result = fetch(task)

3

In [16]:
result

3

https://youtu.be/trhsvOAH0YI?t=9248